In [2]:
import requests
import json
import time
import os
from datetime import datetime
import zipfile
import pandas as pd

class SurveySolutionsDownloader:
    def __init__(self, server_url, username, password, workspace="primary"):
        """
        Initialise le client Survey Solutions
        
        Args:
            server_url (str): URL du serveur Survey Solutions (ex: https://votre-serveur.surveysolutions.org)
            username (str): Nom d'utilisateur API
            password (str): Mot de passe API
            workspace (str): Nom de l'espace de travail (par défaut "primary")
        """
        self.server_url = server_url.rstrip('/')
        self.username = username
        self.password = password
        self.workspace = workspace
        self.session = requests.Session()
        self.token = None
        
    def authenticate(self):
        """Authentification sur l'API Survey Solutions"""
        auth_url = f"{self.server_url}/api/v1/auth/token"
        
        auth_data = {
            "Username": self.username,
            "Password": self.password
        }
        
        try:
            response = self.session.post(auth_url, json=auth_data)
            response.raise_for_status()
            self.token = response.json()["Token"]
            
            # Ajouter le token aux headers pour les requêtes suivantes
            self.session.headers.update({
                "Authorization": f"Bearer {self.token}",
                "Content-Type": "application/json"
            })
            
            print("✅ Authentification réussie")
            return True
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur d'authentification: {e}")
            return False
    
    def get_questionnaires(self):
        """Récupère la liste des questionnaires disponibles"""
        url = f"{self.server_url}/api/v1/questionnaires"
        
        try:
            response = self.session.get(url)
            response.raise_for_status()
            questionnaires = response.json()["Questionnaires"]
            
            print(f"📋 {len(questionnaires)} questionnaire(s) trouvé(s)")
            for q in questionnaires:
                print(f"  - {q['Title']} (ID: {q['QuestionnaireId']}, Version: {q['Version']})")
            
            return questionnaires
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur lors de la récupération des questionnaires: {e}")
            return []
    
    def find_questionnaire_by_title(self, title):
        """Trouve un questionnaire par son titre"""
        questionnaires = self.get_questionnaires()
        
        for q in questionnaires:
            if title.lower() in q['Title'].lower():
                return {
                    'id': q['QuestionnaireId'],
                    'version': q['Version'],
                    'title': q['Title']
                }
        
        print(f"⚠️ Questionnaire '{title}' non trouvé")
        return None
    
    def request_export(self, questionnaire_id, version, export_type="Tabular", 
                      include_meta=True, translation_id=None):
        """
        Demande l'export des données pour un questionnaire
        
        Args:
            questionnaire_id (str): ID du questionnaire
            version (int): Version du questionnaire
            export_type (str): Type d'export ("Tabular", "SPSS", "Stata", "Binary")
            include_meta (bool): Inclure les métadonnées
            translation_id (str): ID de traduction (optionnel)
        """
        url = f"{self.server_url}/api/v1/export/{export_type}/{questionnaire_id}/{version}"
        
        export_data = {
            "ExportType": export_type,
            "QuestionnaireId": questionnaire_id,
            "QuestionnaireVersion": version,
            "IncludeMeta": include_meta
        }
        
        # Ajouter les statuts requis (achevé, approuvé superviseur, approuvé QG)
        export_data["InterviewStatuses"] = [
            "Completed",
            "ApprovedBySupervisor", 
            "ApprovedByHeadquarters"
        ]
        
        if translation_id:
            export_data["TranslationId"] = translation_id
        
        try:
            response = self.session.post(url, json=export_data)
            response.raise_for_status()
            
            job_id = response.json()["JobId"]
            print(f"📤 Export demandé - Job ID: {job_id}")
            return job_id
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur lors de la demande d'export: {e}")
            return None
    
    def check_export_status(self, job_id):
        """Vérifie le statut d'un export"""
        url = f"{self.server_url}/api/v1/export/{job_id}"
        
        try:
            response = self.session.get(url)
            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur lors de la vérification du statut: {e}")
            return None
    
    def download_export(self, job_id, output_folder="downloads"):
        """
        Télécharge un export terminé
        
        Args:
            job_id (str): ID du job d'export
            output_folder (str): Dossier de destination
        """
        url = f"{self.server_url}/api/v1/export/{job_id}/file"
        
        # Créer le dossier de sortie s'il n'existe pas
        os.makedirs(output_folder, exist_ok=True)
        
        try:
            response = self.session.get(url, stream=True)
            response.raise_for_status()
            
            # Extraire le nom du fichier des headers
            filename = f"export_{job_id}.zip"
            if 'content-disposition' in response.headers:
                import re
                cd = response.headers['content-disposition']
                filename_match = re.findall('filename=(.+)', cd)
                if filename_match:
                    filename = filename_match[0].strip('"')
            
            filepath = os.path.join(output_folder, filename)
            
            # Télécharger le fichier
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            print(f"💾 Fichier téléchargé: {filepath}")
            return filepath
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur lors du téléchargement: {e}")
            return None
    
    def wait_for_export(self, job_id, max_wait_time=1800, check_interval=30):
        """
        Attend qu'un export soit terminé
        
        Args:
            job_id (str): ID du job d'export
            max_wait_time (int): Temps d'attente maximum en secondes (30 min par défaut)
            check_interval (int): Intervalle entre les vérifications en secondes
        """
        start_time = time.time()
        
        while time.time() - start_time < max_wait_time:
            status = self.check_export_status(job_id)
            
            if not status:
                return False
            
            export_status = status.get("ExportStatus")
            progress = status.get("Progress", 0)
            
            print(f"📊 Statut: {export_status} - Progression: {progress}%")
            
            if export_status == "Completed":
                print("✅ Export terminé!")
                return True
            elif export_status == "Failed":
                print("❌ Export échoué!")
                return False
            
            time.sleep(check_interval)
        
        print("⏰ Timeout: l'export a pris trop de temps")
        return False
    
    def download_questionnaire_data(self, questionnaire_title, output_folder="downloads"):
        """
        Télécharge les données d'un questionnaire complet
        
        Args:
            questionnaire_title (str): Titre du questionnaire à télécharger
            output_folder (str): Dossier de destination
        """
        print(f"\n🔍 Recherche du questionnaire: {questionnaire_title}")
        
        # Trouver le questionnaire
        questionnaire = self.find_questionnaire_by_title(questionnaire_title)
        if not questionnaire:
            return False
        
        print(f"📝 Questionnaire trouvé: {questionnaire['title']}")
        
        # Demander l'export
        job_id = self.request_export(
            questionnaire['id'], 
            questionnaire['version'],
            export_type="Tabular"  # Format CSV/Excel
        )
        
        if not job_id:
            return False
        
        # Attendre que l'export soit prêt
        if not self.wait_for_export(job_id):
            return False
        
        # Télécharger le fichier
        filepath = self.download_export(job_id, output_folder)
        
        if filepath:
            # Créer un dossier spécifique pour ce questionnaire
            questionnaire_folder = os.path.join(output_folder, questionnaire_title.replace(" ", "_"))
            os.makedirs(questionnaire_folder, exist_ok=True)
            
            # Extraire le fichier ZIP si nécessaire
            if filepath.endswith('.zip'):
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    zip_ref.extractall(questionnaire_folder)
                print(f"📁 Fichiers extraits dans: {questionnaire_folder}")
            
            return True
        
        return False



In [ ]:
def main():
    """Fonction principale pour télécharger les données des enquêtes Tontine et Ordure"""
    
    # ⚠️ CONFIGURATION - Modifiez ces valeurs selon votre installation
    SERVER_URL = "https://votre-serveur.surveysolutions.org"  # Remplacez par votre URL
    USERNAME = "votre-username"  # Remplacez par votre nom d'utilisateur API
    PASSWORD = "votre-password"  # Remplacez par votre mot de passe API
    WORKSPACE = "primary"  # Remplacez par votre workspace si différent
    
    # Dossier de téléchargement avec timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_folder = f"enquete_data_{timestamp}"
    
    print("🚀 Démarrage du téléchargement des données Survey Solutions")
    print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📁 Dossier de sortie: {output_folder}")
    
    # Initialiser le client
    downloader = SurveySolutionsDownloader(SERVER_URL, USERNAME, PASSWORD, WORKSPACE)
    
    # Authentification
    if not downloader.authenticate():
        print("❌ Impossible de s'authentifier. Vérifiez vos identifiants.")
        return
    
    # Liste des questionnaires à télécharger
    questionnaires = ["Tontine", "Ordure"]
    
    success_count = 0
    
    for questionnaire in questionnaires:
        print(f"\n{'='*50}")
        print(f"📋 Téléchargement: {questionnaire}")
        print(f"{'='*50}")
        
        if downloader.download_questionnaire_data(questionnaire, output_folder):
            success_count += 1
            print(f"✅ {questionnaire} téléchargé avec succès!")
        else:
            print(f"❌ Échec du téléchargement de {questionnaire}")
    
    print(f"\n{'='*50}")
    print(f"📊 RÉSUMÉ")
    print(f"{'='*50}")
    print(f"✅ Réussis: {success_count}/{len(questionnaires)}")
    print(f"📁 Données sauvegardées dans: {output_folder}")
    
    if success_count == len(questionnaires):
        print("🎉 Tous les téléchargements ont réussi!")
    else:
        print("⚠️ Certains téléchargements ont échoué. Vérifiez les logs ci-dessus.")

if __name__ == "__main__":
    main()

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Script pour générer un fichier Excel avec la structure administrative du Cameroun
Auteur: Assistant IA
Date: Juillet 2025
"""

import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
import os

def create_cameroon_admin_data():
    """
    Crée les données administratives du Cameroun
    Returns: list de tuples (Région, Département, Arrondissement, Quartier)
    """
    data = [
        # REGION CENTRE
        ("Centre", "Mfoundi", "Yaoundé I", "Essos"),
        ("Centre", "Mfoundi", "Yaoundé I", "Centre-ville"),
        ("Centre", "Mfoundi", "Yaoundé I", "Nlongkak"),
        ("Centre", "Mfoundi", "Yaoundé I", "Mokolo"),
        ("Centre", "Mfoundi", "Yaoundé I", "Mvog-Ada"),
        ("Centre", "Mfoundi", "Yaoundé II", "Tsinga"),
        ("Centre", "Mfoundi", "Yaoundé II", "Elig-Edzoa"),
        ("Centre", "Mfoundi", "Yaoundé II", "Melen"),
        ("Centre", "Mfoundi", "Yaoundé II", "Messame-Ndongo"),
        ("Centre", "Mfoundi", "Yaoundé III", "Efoulan"),
        ("Centre", "Mfoundi", "Yaoundé III", "Mvog-Mbi"),
        ("Centre", "Mfoundi", "Yaoundé III", "Nkolbisson"),
        ("Centre", "Mfoundi", "Yaoundé III", "Ahala"),
        ("Centre", "Mfoundi", "Yaoundé IV", "Kondengui"),
        ("Centre", "Mfoundi", "Yaoundé IV", "Ekounou"),
        ("Centre", "Mfoundi", "Yaoundé IV", "Nkolmesseng"),
        ("Centre", "Mfoundi", "Yaoundé V", "Ngousso"),
        ("Centre", "Mfoundi", "Yaoundé V", "Emombo"),
        ("Centre", "Mfoundi", "Yaoundé V", "Mvog-Betsi Zoo"),
        ("Centre", "Mfoundi", "Yaoundé VI", "Biyem-Assi"),
        ("Centre", "Mfoundi", "Yaoundé VI", "Nkoldongo"),
        ("Centre", "Mfoundi", "Yaoundé VI", "Mendong"),
        ("Centre", "Mfoundi", "Yaoundé VII", "Nkolso"),
        ("Centre", "Mfoundi", "Yaoundé VII", "Oyom-Abang"),
        ("Centre", "Mélé", "Mbalmayo", "Centre"),
        ("Centre", "Mélé", "Mbalmayo", "Elat"),
        ("Centre", "Mélé", "Mbalmayo", "Nguibassal"),
        ("Centre", "Nyong-et-Kellé", "Eseka", "Centre"),
        ("Centre", "Nyong-et-Kellé", "Eseka", "Makak"),
        ("Centre", "Nyong-et-Kellé", "Botmakak", "Centre"),
        ("Centre", "Lékie", "Monatélé", "Centre"),
        ("Centre", "Lékie", "Obala", "Centre"),
        ("Centre", "Lékie", "Saa", "Centre"),
        ("Centre", "Haute-Sanaga", "Nanga-Eboko", "Centre"),
        ("Centre", "Mbam-et-Inoubou", "Bafia", "Centre"),
        ("Centre", "Mbam-et-Kim", "Ntui", "Centre"),
        
        # REGION LITTORAL
        ("Littoral", "Wouri", "Douala I", "Akwa"),
        ("Littoral", "Wouri", "Douala I", "Bonanjo"),
        ("Littoral", "Wouri", "Douala I", "Deido"),
        ("Littoral", "Wouri", "Douala I", "Bali"),
        ("Littoral", "Wouri", "Douala II", "New Bell"),
        ("Littoral", "Wouri", "Douala II", "Bassa"),
        ("Littoral", "Wouri", "Douala II", "Ndog-Bong"),
        ("Littoral", "Wouri", "Douala III", "Makepe"),
        ("Littoral", "Wouri", "Douala III", "Logpom"),
        ("Littoral", "Wouri", "Douala III", "Ndogbong"),
        ("Littoral", "Wouri", "Douala IV", "Bonaberi"),
        ("Littoral", "Wouri", "Douala IV", "Bessengue"),
        ("Littoral", "Wouri", "Douala IV", "Cite SIC"),
        ("Littoral", "Wouri", "Douala V", "Kotto"),
        ("Littoral", "Wouri", "Douala V", "Bepanda"),
        ("Littoral", "Wouri", "Douala V", "Nyalla"),
        ("Littoral", "Wouri", "Douala VI", "PK8"),
        ("Littoral", "Wouri", "Douala VI", "PK11"),
        ("Littoral", "Wouri", "Douala VI", "Yassa"),
        ("Littoral", "Sanaga-Maritime", "Edéa", "Centre"),
        ("Littoral", "Sanaga-Maritime", "Edéa", "Dizangué"),
        ("Littoral", "Sanaga-Maritime", "Pouma", "Centre"),
        ("Littoral", "Mungo", "Nkongsamba", "Centre"),
        ("Littoral", "Mungo", "Melong", "Centre"),
        ("Littoral", "Mungo", "Loum", "Centre"),
        ("Littoral", "Nkam", "Yabassi", "Centre"),
        
        # REGION OUEST
        ("Ouest", "Mifi", "Bafoussam I", "Centre"),
        ("Ouest", "Mifi", "Bafoussam I", "Tamdja"),
        ("Ouest", "Mifi", "Bafoussam I", "Tougang"),
        ("Ouest", "Mifi", "Bafoussam II", "Tchoumba"),
        ("Ouest", "Mifi", "Bafoussam II", "Djeleng"),
        ("Ouest", "Mifi", "Bafoussam III", "Famla"),
        ("Ouest", "Mifi", "Bafoussam III", "Banengo"),
        ("Ouest", "Noun", "Foumban", "Centre"),
        ("Ouest", "Noun", "Foumban", "Njimom"),
        ("Ouest", "Noun", "Foumbot", "Centre"),
        ("Ouest", "Bamboutos", "Mbouda", "Centre"),
        ("Ouest", "Bamboutos", "Galim", "Centre"),
        ("Ouest", "Bamboutos", "Babadjou", "Centre"),
        ("Ouest", "Menoua", "Dschang", "Centre"),
        ("Ouest", "Menoua", "Dschang", "Fongo-Tongo"),
        ("Ouest", "Menoua", "Fokoué", "Centre"),
        ("Ouest", "Menoua", "Nkong-Ni", "Centre"),
        ("Ouest", "Haut-Nkam", "Bafang", "Centre"),
        ("Ouest", "Haut-Nkam", "Kekem", "Centre"),
        ("Ouest", "Ndé", "Bangangté", "Centre"),
        ("Ouest", "Ndé", "Bazou", "Centre"),
        ("Ouest", "Koung-Khi", "Bandjoun", "Centre"),
        ("Ouest", "Hauts-Plateaux", "Baham", "Centre"),
        
        # REGION NORD-OUEST
        ("Nord-Ouest", "Mezam", "Bamenda I", "Commercial Avenue"),
        ("Nord-Ouest", "Mezam", "Bamenda I", "Cow Street"),
        ("Nord-Ouest", "Mezam", "Bamenda I", "City Council"),
        ("Nord-Ouest", "Mezam", "Bamenda II", "Mulang"),
        ("Nord-Ouest", "Mezam", "Bamenda II", "Mankon"),
        ("Nord-Ouest", "Mezam", "Bamenda III", "Nkwen"),
        ("Nord-Ouest", "Mezam", "Bamenda III", "Mendankwe"),
        ("Nord-Ouest", "Mezam", "Tubah", "Centre"),
        ("Nord-Ouest", "Momo", "Mbengwi", "Centre"),
        ("Nord-Ouest", "Momo", "Batibo", "Centre"),
        ("Nord-Ouest", "Momo", "Njikwa", "Centre"),
        ("Nord-Ouest", "Boyo", "Fundong", "Centre"),
        ("Nord-Ouest", "Boyo", "Kumbo", "Centre"),
        ("Nord-Ouest", "Bui", "Kumbo", "Centre"),
        ("Nord-Ouest", "Bui", "Noni", "Centre"),
        ("Nord-Ouest", "Donga-Mantung", "Nkambe", "Centre"),
        ("Nord-Ouest", "Donga-Mantung", "Misaje", "Centre"),
        ("Nord-Ouest", "Menchum", "Wum", "Centre"),
        ("Nord-Ouest", "Menchum", "Furu-Awa", "Centre"),
        ("Nord-Ouest", "Ng-Ketunjia", "Ndop", "Centre"),
        ("Nord-Ouest", "Ng-Ketunjia", "Babessi", "Centre"),
        
        # REGION SUD-OUEST
        ("Sud-Ouest", "Fako", "Buea", "Great Soppo"),
        ("Sud-Ouest", "Fako", "Buea", "Molyko"),
        ("Sud-Ouest", "Fako", "Buea", "Mile 16"),
        ("Sud-Ouest", "Fako", "Limbe I", "Down Beach"),
        ("Sud-Ouest", "Fako", "Limbe I", "Church Street"),
        ("Sud-Ouest", "Fako", "Limbe II", "Mile 2"),
        ("Sud-Ouest", "Fako", "Limbe II", "Motowo"),
        ("Sud-Ouest", "Fako", "Tiko", "Centre"),
        ("Sud-Ouest", "Fako", "Muyuka", "Centre"),
        ("Sud-Ouest", "Meme", "Kumba I", "Fiango"),
        ("Sud-Ouest", "Meme", "Kumba I", "Mbonge Road"),
        ("Sud-Ouest", "Meme", "Kumba II", "Kossala"),
        ("Sud-Ouest", "Meme", "Kumba II", "Buea Road"),
        ("Sud-Ouest", "Meme", "Mbonge", "Centre"),
        ("Sud-Ouest", "Meme", "Konye", "Centre"),
        ("Sud-Ouest", "Kupe-Manenguba", "Bangem", "Centre"),
        ("Sud-Ouest", "Kupe-Manenguba", "Nguti", "Centre"),
        ("Sud-Ouest", "Lebialem", "Menji", "Centre"),
        ("Sud-Ouest", "Lebialem", "Wabane", "Centre"),
        ("Sud-Ouest", "Manyu", "Mamfe", "Centre"),
        ("Sud-Ouest", "Manyu", "Eyumojock", "Centre"),
        ("Sud-Ouest", "Ndian", "Mundemba", "Centre"),
        ("Sud-Ouest", "Ndian", "Toko", "Centre"),
        
        # REGION NORD
        ("Nord", "Bénoué", "Garoua I", "Centre"),
        ("Nord", "Bénoué", "Garoua I", "Dougoï"),
        ("Nord", "Bénoué", "Garoua II", "Kollere"),
        ("Nord", "Bénoué", "Garoua II", "Ouro Kessoum"),
        ("Nord", "Bénoué", "Garoua III", "Djamboutou"),
        ("Nord", "Bénoué", "Pitoa", "Centre"),
        ("Nord", "Bénoué", "Lagdo", "Centre"),
        ("Nord", "Faro", "Tignère", "Centre"),
        ("Nord", "Faro", "Banyo", "Centre"),
        ("Nord", "Mayo-Louti", "Guider", "Centre"),
        ("Nord", "Mayo-Louti", "Figuil", "Centre"),
        ("Nord", "Mayo-Rey", "Tcholliré", "Centre"),
        ("Nord", "Mayo-Rey", "Rey-Bouba", "Centre"),
        
        # REGION ADAMAOUA
        ("Adamaoua", "Adamaoua", "Ngaoundéré I", "Centre"),
        ("Adamaoua", "Adamaoua", "Ngaoundéré I", "Haoussa"),
        ("Adamaoua", "Adamaoua", "Ngaoundéré II", "Dang"),
        ("Adamaoua", "Adamaoua", "Ngaoundéré II", "Sabongari"),
        ("Adamaoua", "Adamaoua", "Ngaoundéré III", "Baladji"),
        ("Adamaoua", "Djérem", "Tibati", "Centre"),
        ("Adamaoua", "Djérem", "Ngaoui", "Centre"),
        ("Adamaoua", "Faro-et-Déo", "Tignère", "Centre"),
        ("Adamaoua", "Faro-et-Déo", "Kontcha", "Centre"),
        ("Adamaoua", "Mayo-Banyo", "Banyo", "Centre"),
        ("Adamaoua", "Mbéré", "Meiganga", "Centre"),
        ("Adamaoua", "Mbéré", "Ngaounda", "Centre"),
        ("Adamaoua", "Vina", "Nganha", "Centre"),
        ("Adamaoua", "Vina", "Belel", "Centre"),
        
        # REGION EXTREME-NORD
        ("Extrême-Nord", "Diamaré", "Maroua I", "Centre"),
        ("Extrême-Nord", "Diamaré", "Maroua I", "Dougoy"),
        ("Extrême-Nord", "Diamaré", "Maroua II", "Domayo"),
        ("Extrême-Nord", "Diamaré", "Maroua III", "Pitoaré"),
        ("Extrême-Nord", "Diamaré", "Bogo", "Centre"),
        ("Extrême-Nord", "Mayo-Danay", "Yagoua", "Centre"),
        ("Extrême-Nord", "Mayo-Danay", "Waza", "Centre"),
        ("Extrême-Nord", "Mayo-Kani", "Kaélé", "Centre"),
        ("Extrême-Nord", "Mayo-Kani", "Mindif", "Centre"),
        ("Extrême-Nord", "Mayo-Sava", "Mora", "Centre"),
        ("Extrême-Nord", "Mayo-Sava", "Kolofata", "Centre"),
        ("Extrême-Nord", "Mayo-Tsanaga", "Mokolo", "Centre"),
        ("Extrême-Nord", "Mayo-Tsanaga", "Koza", "Centre"),
        ("Extrême-Nord", "Logone-et-Chari", "Kousséri", "Centre"),
        ("Extrême-Nord", "Logone-et-Chari", "Fotokol", "Centre"),
        
        # REGION EST
        ("Est", "Haut-Nyong", "Abong-Mbang", "Centre"),
        ("Est", "Haut-Nyong", "Doumé", "Centre"),
        ("Est", "Haut-Nyong", "Messondo", "Centre"),
        ("Est", "Kadey", "Batouri", "Centre"),
        ("Est", "Kadey", "Kenzou", "Centre"),
        ("Est", "Lom-et-Djérem", "Bétaré-Oya", "Centre"),
        ("Est", "Lom-et-Djérem", "Garoua-Boulaï", "Centre"),
        ("Est", "Boumba-et-Ngoko", "Yokadouma", "Centre"),
        ("Est", "Boumba-et-Ngoko", "Moloundou", "Centre"),
        
        # REGION SUD
        ("Sud", "Dja-et-Lobo", "Sangmélima", "Centre"),
        ("Sud", "Dja-et-Lobo", "Djoum", "Centre"),
        ("Sud", "Dja-et-Lobo", "Mintom", "Centre"),
        ("Sud", "Mvila", "Ebolowa I", "Centre"),
        ("Sud", "Mvila", "Ebolowa I", "Angalé"),
        ("Sud", "Mvila", "Ebolowa II", "Ngovayang"),
        ("Sud", "Mvila", "Mengong", "Centre"),
        ("Sud", "Océan", "Kribi I", "Centre"),
        ("Sud", "Océan", "Kribi I", "Londji"),
        ("Sud", "Océan", "Kribi II", "Akom II"),
        ("Sud", "Océan", "Campo", "Centre"),
        ("Sud", "Vallée-du-Ntem", "Ambam", "Centre"),
        ("Sud", "Vallée-du-Ntem", "Olamze", "Centre"),
    ]
    
    return data

def format_excel_file(filename):
    """
    Applique le formatage au fichier Excel
    """
    from openpyxl import load_workbook
    
    # Charger le classeur
    wb = load_workbook(filename)
    ws = wb.active
    
    # Définir les styles
    header_font = Font(bold=True, color="FFFFFF", size=12)
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    center_alignment = Alignment(horizontal="center", vertical="center")
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Formater les en-têtes
    for col in range(1, 5):  # Colonnes A à D
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_alignment
        cell.border = border
    
    # Ajuster la largeur des colonnes
    column_widths = [15, 25, 30, 25]  # Région, Département, Arrondissement, Quartier
    for i, width in enumerate(column_widths, 1):
        column_letter = ws.cell(row=1, column=i).column_letter
        ws.column_dimensions[column_letter].width = width
    
    # Appliquer les bordures à toutes les cellules avec données
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=4):
        for cell in row:
            cell.border = border
    
    # Alterner les couleurs des lignes
    light_fill = PatternFill(start_color="F8F9FA", end_color="F8F9FA", fill_type="solid")
    for row_num in range(2, ws.max_row + 1):
        if row_num % 2 == 0:  # Lignes paires
            for col in range(1, 5):
                ws.cell(row=row_num, column=col).fill = light_fill
    
    # Sauvegarder
    wb.save(filename)

def main():
    """
    Fonction principale pour créer le fichier Excel
    """
    print("🇨🇲 Génération du fichier Excel - Structure Administrative du Cameroun")
    print("=" * 70)
    
    # Créer les données
    print("📊 Création des données administratives...")
    data = create_cameroon_admin_data()
    
    # Créer le DataFrame
    df = pd.DataFrame(data, columns=["Région", "Département", "Arrondissement", "Quartier"])
    
    # Afficher les statistiques
    print(f"✅ Données créées : {len(df)} enregistrements")
    print(f"📍 Régions : {df['Région'].nunique()}")
    print(f"🏛️  Départements : {df['Département'].nunique()}")
    print(f"🏢 Arrondissements : {df['Arrondissement'].nunique()}")
    print(f"🏘️  Quartiers : {df['Quartier'].nunique()}")
    print()
    
    # Sauvegarder en Excel
    filename = "Structure_Administrative_Cameroun.xlsx"
    print(f"💾 Sauvegarde du fichier : {filename}")
    
    # Méthode 1 : Utilisation de pandas (simple)
    df.to_excel(filename, index=False, sheet_name="Structure Administrative")
    
    # Méthode 2 : Formatage avancé avec openpyxl
    print("🎨 Application du formatage...")
    format_excel_file(filename)
    
    # Afficher l'aperçu
    print("\n📋 Aperçu des premières lignes :")
    print(df.head(10).to_string(index=False))
    
    print(f"\n✅ Fichier Excel créé avec succès : {os.path.abspath(filename)}")
    print(f"📁 Taille du fichier : {os.path.getsize(filename)} bytes")

def display_summary():
    """
    Affiche un résumé par région
    """
    data = create_cameroon_admin_data()
    df = pd.DataFrame(data, columns=["Région", "Département", "Arrondissement", "Quartier"])
    
    print("\n📊 RÉSUMÉ PAR RÉGION :")
    print("=" * 50)
    
    summary = df.groupby("Région").agg({
        "Département": "nunique",
        "Arrondissement": "nunique", 
        "Quartier": "count"
    }).rename(columns={
        "Département": "Départements",
        "Arrondissement": "Arrondissements",
        "Quartier": "Quartiers"
    })
    
    print(summary.to_string())

if __name__ == "__main__":
    # Vérifier les dépendances
    try:
        import pandas as pd
        import openpyxl
        print("✅ Toutes les dépendances sont installées")
    except ImportError as e:
        print("❌ Dépendance manquante :", str(e))
        print("\n📦 Pour installer les dépendances, exécutez :")
        print("pip install pandas openpyxl")
        exit(1)
    
    # Exécuter le script principal
    main()
    
    # Afficher le résumé
    display_summary()
    
    print("\n🎉 Script terminé avec succès !")

✅ Toutes les dépendances sont installées
🇨🇲 Génération du fichier Excel - Structure Administrative du Cameroun
📊 Création des données administratives...
✅ Données créées : 193 enregistrements
📍 Régions : 10
🏛️  Départements : 56
🏢 Arrondissements : 133
🏘️  Quartiers : 89

💾 Sauvegarde du fichier : Structure_Administrative_Cameroun.xlsx
🎨 Application du formatage...

📋 Aperçu des premières lignes :
Région Département Arrondissement       Quartier
Centre     Mfoundi      Yaoundé I          Essos
Centre     Mfoundi      Yaoundé I   Centre-ville
Centre     Mfoundi      Yaoundé I       Nlongkak
Centre     Mfoundi      Yaoundé I         Mokolo
Centre     Mfoundi      Yaoundé I       Mvog-Ada
Centre     Mfoundi     Yaoundé II         Tsinga
Centre     Mfoundi     Yaoundé II     Elig-Edzoa
Centre     Mfoundi     Yaoundé II          Melen
Centre     Mfoundi     Yaoundé II Messame-Ndongo
Centre     Mfoundi    Yaoundé III        Efoulan

✅ Fichier Excel créé avec succès : c:\Users\LK\Desktop\GItH